In [1]:
!pip install simpletransformers
!pip install datasets

     |████████████████████████████████| 221 kB 4.9 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 1.7 MB 9.6 MB/s 
     |████████████████████████████████| 76 kB 3.9 MB/s 
     |████████████████████████████████| 264 kB 25.9 MB/s 
     |████████████████████████████████| 124 kB 32.9 MB/s 
     |████████████████████████████████| 7.9 MB 19.7 MB/s 
     |████████████████████████████████| 1.2 MB 56.2 MB/s 
     |████████████████████████████████| 2.6 MB 50.0 MB/s 
     |████████████████████████████████| 3.3 MB 49.9 MB/s 
     |████████████████████████████████| 636 kB 40.4 MB/s 
     |████████████████████████████████| 895 kB 41.1 MB/s 
     |████████████████████████████████| 133 kB 66.5 MB/s 
     |████████████████████████████████| 188 kB 51.6 MB/s 
     |████████████████████████████████| 170 kB 61.6 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 118 kB 61.9 MB/s 
     |████████████████

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!tar -zxvf /content/drive/MyDrive/Models/model_with_key.tar.gz

outputs/best_model/
outputs/best_model/training_args.bin
outputs/best_model/spiece.model
outputs/best_model/optimizer.pt
outputs/best_model/eval_results.txt
outputs/best_model/config.json
outputs/best_model/tokenizer_config.json
outputs/best_model/pytorch_model.bin
outputs/best_model/scheduler.pt
outputs/best_model/special_tokens_map.json
outputs/best_model/model_args.json
outputs/best_model/added_tokens.json


In [4]:
from simpletransformers.t5 import T5Model

In [5]:
t5_model_directory = "outputs/best_model"

In [6]:
# class for generating question
class QuestionGenerator():

  def __init__(self):
    ''' Initializes the class to generate a MCQ given a context and an answer '''
    self.model = T5Model("t5", t5_model_directory)
  

  def get_processed_input(self, context, key):
    ''' Returns processed input with the required tokens '''
    return 'generate_question: {0} answer: {1}'.format(context, key)


  def get_predictions(self, context, key):
    ''' Generates the question from the context '''
    to_predict = [self.get_processed_input(context, key)]   
    predictions = self.model.predict(to_predict)    
    if len(predictions) == 1:
      return predictions[0]
    else:
      raise Exception('Not able to generate')


  def get_mcq(self, context, key):
    ''' Call this method to get the final MCQ for the provided context and key '''
    question = self.get_predictions(context, key)
    return question

In [13]:
# class for generating answers
from transformers import T5ForConditionalGeneration, T5Tokenizer
class AnswerGenerator():
  
  def __init__(self):
    base_model = 't5-base'
    self.tokenizer = T5Tokenizer.from_pretrained(base_model)
    self.model = T5ForConditionalGeneration.from_pretrained(base_model)

  def get_answer_for_question(self, question, context):
    input = 'question: %s context: %s' % (question, context)
    features = self.tokenizer(input, return_tensors='pt')
    input_ids = features['input_ids']
    attention_mask = features['attention_mask']
    outcome = self.model.generate(input_ids = input_ids, attention_mask = attention_mask, max_length=255)
    answer = self.tokenizer.decode(outcome[0]).replace('<pad>', '').replace('</s>', '').strip()
    return answer


In [8]:
import pandas as pd

In [9]:
# loading dataset
from datasets import load_dataset
raw_datasets = load_dataset('squad')

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


In [10]:
# Preprocessing validation dataset and preparing list of dictionary for evaluation
validation_list = list()
for i in range(len(raw_datasets['validation'])):
  new_record = dict()
  new_record['context'] = raw_datasets['validation'][i]['context']
  new_record['answer'] = raw_datasets['validation'][i]['answers']['text'][0]
  validation_list.append(new_record)

In [14]:
# Evaluation technique
# Generate question using our trained t5 model
# Feed the question to t5-base model
# compare the t5-base model answer with input key to our t5 model
class Evaluate():
  def __init__(self):
    self.ques_generator = QuestionGenerator()
    self.ans_generator = AnswerGenerator()

  def eval(self):
    correct = 0
    for record in validation_list[:100]:
      context = record['context']
      key = record['answer']
      question = self.ques_generator.get_mcq(context, key)
      answer = self.ans_generator.get_answer_for_question(question, context)
      if key == answer:
        correct += 1
    return correct

In [ ]:
correct_preds = Evaluate().eval()

In [16]:
correct_preds

34